# Mix Classes=Categories
version: 1

info: 
- mix/fusion of classes into other classes

author: nuno costa

In [54]:
from annotate_v5 import *
import platform 
import numpy as np
import pandas as pd
from IPython.display import Image, display
import copy
import os
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib.image import imread
from matplotlib.patches import Rectangle
import random


#Define root dir dependent on OS
rdir='D:/external_datasets/' 
if str(platform.platform()).find('linux')>-1: rdir=rdir.replace('D:/','/mnt/d/')
print('OS: {}'.format(platform.platform()))
print('root dir: {}'.format(rdir))

OS: Windows-10-10.0.20251-SP0
root dir: D:/external_datasets/


In [55]:
molajson =  json.load(open(rdir+'MOLA/annotations/split_mola_fix_equal/test.json'))

In [56]:
for k in molajson:
    print(k, len(molajson[k]))

info 5
licenses 9
categories 1261
videos 1488
images 177936
tracks 8132
segment_info 0
annotations 133641
datasets 2


## 1. Import ids
#### #NOTE: work with ids and index so you can use numpy for faster operations

In [57]:
# datasets name and id
dset_l=[]
dset_l_id=[]
for d in molajson['datasets']:
    dset_l.append(d['name'])
    dset_l_id.append(d['id'])
print(dset_l, dset_l_id)

['COCO', 'TAO'] [1, 2]


In [58]:
# categories name and id
cat_l=[]
cat_l_id=[]
cat_l_dset=[]
for c in molajson['categories']:
    cat_l.append(c['name'])
    cat_l_id.append(c['id'])
    cat_l_dset.append(dset_l[c['dataset']-1]) # dset_l index is same as id-1
#print(cat_l_id)

In [59]:
# images filepath and id
img_l=[]
img_l_id=[]
for c in molajson['images']:
    img_l.append(c['file_name'])
    img_l_id.append(c['id'])

In [60]:
# annotations category_id, image_id, bbox, and dataset
ann_catid=[]
ann_imgid=[]
ann_bbox=[]
ann_dset=[]
for an in tqdm(molajson['annotations']):
    ann_catid.append(an['category_id'])
    ann_imgid.append(an['image_id'])
    ann_bbox.append(an['bbox'])
    ann_dset.append(an['dataset'])

100%|████████████████████████████████████████████████████████████████████| 133641/133641 [00:00<00:00, 835440.05it/s]


## 2. Find mixers cat_ids
mixers example
categories= [{name:cow, id:1, dataset:1},...,{name:cow, id:200, dataset:2},...,{name:cow, id:101, dataset:3}]}

In [61]:
#mixers #TODO: SORT alphabetically
mixers_l=[]
mixers_l_catid=[]
mixers_l_catdset=[]
mixer_method="all_names"
if mixer_method=="all_names": #Do for all category names, even with equal 
    mixers_l=cat_l
    mixers_l_catid=[[id] for id in cat_l_id]
    mixers_l_catdset=[[dset] for dset in cat_l_dset]
    

print(mixers_l[0:5])
print(mixers_l_catid[0:5])
print(mixers_l_catdset[0:5])
print(len(mixers_l))
print(len(mixers_l_catid))
print(len(mixers_l_catdset))

['person', 'bicycle', 'car', 'motorcycle', 'airplane']
[[1], [2], [3], [4], [5]]
[['COCO'], ['COCO'], ['COCO'], ['COCO'], ['COCO']]
1261
1261
1261


In [62]:
# get annotations mixers
ann_catid_np=np.array(ann_catid)
ann_imgid_np=np.array(ann_imgid)
ann_bbox_np=np.array(ann_bbox)
ann_dset_np=np.array(ann_dset)
mixers_l_imgid=[]
mixers_l_bbox=[]
mixers_l_dset=[]
for catids in tqdm(mixers_l_catid):
    l_imgid=[]
    l_bbox=[]
    l_dset=[]
    for catid in catids:
        ann_idx = np.where(ann_catid_np==catid)[0].tolist() #annotation index of ids
        l_imgid.append(ann_imgid_np[ann_idx].tolist())
        l_bbox.append(ann_bbox_np[ann_idx].tolist())
        l_dset.append(ann_dset_np[ann_idx].tolist())
    mixers_l_imgid.append(l_imgid)
    mixers_l_bbox.append(l_bbox)
    mixers_l_dset.append(l_dset)

100%|██████████████████████████████████████████████████████████████████████████| 1261/1261 [00:00<00:00, 3915.64it/s]


## 3. Classes|categories to fix w/ EXCEL report

In [63]:
#INIT VARS
classtomix_l=[]
classtomix_l_catid=[]
method="" #save_images to save images and report for manual inspection
datadir="mixers"
folder=mixer_method+'/' #folder
showimage=False #show images
startidx=0 # start index of image to save from each dataset
imgnr=5 # total number of images to save from each dataset
imgstep='random' # step between images: int | 'random' - int steps between images; 'rand' gets random list
#paths
path=os.path.join(rdir,datadir,folder) #path to folder
assure_path_exists(path)
excelpath=path+mixer_method+"_classtomix_report.xlsx"#path to excel

In [64]:
#METHODS
if method=="save_images": # save images and excel to folder for manual edit
    df=pd.DataFrame({'mixers_l': mixers_l,'mixers_l_catid': mixers_l_catid, 'mixers_l_catdset': mixers_l_catdset, 'classtomix_l': np.nan, 'classtomix_l_catid':np.nan, 'rules':np.nan })
    df.loc[0, 'rules']="To fix classes: 1) You need to fill the column classtomix_l and/or classtomix_l_catid with the information from the respective mixer columns; 2) When copy/pasting or changing, make sure the same structure maintains:  ['car', 'carrot'], [3, 52], beware of the spaces ['car', '  and always maintain the first class in the list;  3) You have 3 possibilities of filling the columns : 1-the 2 columns empty, meaning the row will not be used for classtomix; 2-only one column empty, e.g. fill the classtotix_l row with the class labels from mixers_l, then during the importing the classtomix_l_catid is filled, and vice-versa; 3-If you want to change the name of the first class in the list,e.g ['car', 'carrot'] for ['automobile', 'carrot'] you need to provide the ids to classtomix_l_catid."
    df['annotations_missing'] = np.empty((len(df), 0)).tolist()
    df['images_missing'] = np.empty((len(df), 0)).tolist()
    #save image for each mixer
    for i, mixer in enumerate(tqdm(mixers_l)): #run for each mixer category
        firstclass=mixer
        if isinstance(firstclass, list): firstclass=firstclass[0] #first class
        print('\n>> '+firstclass+'...') #class
        classpath=os.path.join(path, firstclass) # path to folder for images of  firstclass
        classpath=parse_path(classpath)+'/' #make it a folder
        assure_path_exists(classpath)
        df=save_imgs(df, rdir, classpath, i, dset_l, mixers_l, mixers_l_catid, mixers_l_bbox, mixers_l_dset,
              mixers_l_imgid, img_l, img_l_id, startidx=startidx, imgnr=imgnr, imgstep=imgstep, showimage=showimage)    
    df.to_excel(excelpath, index=False)   

In [65]:
#IMPORT EXCEL MANUAL EDIT #WARNING: CHECK EXCEL FIRST
df=pd.read_excel(excelpath)
classtomix_df=df.loc[:,'classtomix_l']
classtomix_df_catid=df.loc[:,'classtomix_l_catid']
display(df)
# PARSE COLUMNS TO FIX
classtomix_l=classtomix_df.tolist()
classtomix_l_catid=classtomix_df_catid.tolist()
#convert strings to lists
for icl, cl in enumerate(classtomix_l): 
    if isinstance(classtomix_l[icl], str): classtomix_l[icl]=convert_unicode(classtomix_l[icl], method='liststr')
    if isinstance(classtomix_l_catid[icl], str): classtomix_l_catid[icl]=convert_unicode(classtomix_l_catid[icl], method='listnum')
#parse the columns based on the rules    
for ic, classes in enumerate(classtomix_df):
    #1. if only classtomix_l_catid empty - get 
    if not pd.isnull(classtomix_df.iloc[ic]) and pd.isnull(classtomix_df_catid.iloc[ic]):
        if len(classtomix_df.dropna()) == 2: # if only 2 class get remaining ids
            chosed_id=np.array([x for x in classtomix_l_catid if str(x) != 'nan' and str(x) !='[]'][0])
            remain_id=np.setdiff1d(np.array(cat_l_id), chosed_id)
            classtomix_l_catid[ic]=remain_id.tolist()
    #2. if only classtomix_l empty - raise
    if pd.isnull(classtomix_df.iloc[ic]) and not pd.isnull(classtomix_df_catid.iloc[ic]): 
        raise RuntimeError('only classtomix_l empty')

,mixers_l,mixers_l_catid,mixers_l_catdset,classtomix_l,classtomix_l_catid,rules,annotations_missing,images_missing,Unnamed: 8,aggressive_keywords,notes
0,person,[1],['COCO'],aggressive,"[43, 44, 45, 216, 256, 613, 675, 707, 905, 979...",To fix classes: 1) You need to fill the column...,[0],[0],NaN,knife,Só adicionei ids com annotations
1,bicycle,[2],['COCO'],nonaggressive,NaN,NaN,[0],[0],NaN,weapon,NaN
2,car,[3],['COCO'],NaN,NaN,NaN,[0],[0],NaN,gun,NaN
3,motorcycle,[4],['COCO'],NaN,NaN,NaN,[0],[0],NaN,bat,NaN
4,airplane,[5],['COCO'],NaN,NaN,NaN,[0],[0],NaN,fork,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1256,yacht,[1305],['TAO'],NaN,NaN,NaN,[1],[0],NaN,NaN,NaN
1257,yak,[1306],['TAO'],NaN,NaN,NaN,[1],[0],NaN,NaN,NaN
1258,yogurt,[1307],['TAO'],NaN,NaN,NaN,[1],[0],NaN,NaN,NaN
1259,yoke_(animal_equipment),[1308],['TAO'],NaN,NaN,NaN,[1],[0],NaN,NaN,NaN


In [66]:
print('>> Make sure everything is correct: 1.Drop NaN if exist, but make sure the index is the same for the two! \n')
fixempty=True
if fixempty:
    classtomix_l=[x for x in classtomix_l if str(x) != 'nan' and str(x) !='[]']
    classtomix_l_catid=[x for x in classtomix_l_catid if str(x) != 'nan' and str(x) !='[]']
print(classtomix_l)
print(classtomix_l_catid)

>> Make sure everything is correct: 1.Drop NaN if exist, but make sure the index is the same for the two! 

[['aggressive'], ['nonaggressive']]
[[43, 44, 45, 216, 256, 613, 675, 707, 905, 979, 1022, 1246, 1300], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 117, 118, 119, 120, 121, 122, 123, 124, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 160, 162, 163, 164, 165, 166, 167, 168, 169, 170, 172, 173, 174, 176, 177, 178, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 

## 4. Mix classes

In [67]:
# slow # newjson=copy.deepcopy(molajson) #do deepcopy to compare
# fast
newjson={'categories':[],'annotations':[] }
newjson['categories']=copy.copy(molajson['categories'])
newjson['annotations']=copy.copy(molajson['annotations'])

In [68]:
classtomix_l_catidx=[[cat_l_id.index(id) for id in id_l] for id_l in classtomix_l_catid]
#print(classtomix_l_catidx) # they should be less one, becacuse it is ordered
print((classtomix_l_catidx))

[[42, 43, 44, 200, 238, 583, 644, 674, 871, 944, 985, 1197, 1251], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211,

#### Change molajson['categories']: [{name: , id: }]  
=>  1. use first index cat id; 2. change name and change id;  remove the other categories (!!!Without ordering again the category id!!!)

In [69]:
# CHANGE NAME  & GET REMOVE List
keepidx_l=[]
keepid_l=[]
firstidx=0 # get first category id
for i,id_l in enumerate(tqdm(classtomix_l_catid)): #for each classtomix
    firstcatid=id_l[firstidx] # #category id 
    firstcatidx=classtomix_l_catidx[i][firstidx]# get cat index of first catid
    if isinstance(classtomix_l[i], list): newjson['categories'][firstcatidx]['name']=classtomix_l[i][firstidx] #change name of first id 
    else: newjson['categories'][firstcatidx]['name']=classtomix_l[i]
    assert newjson['categories'][firstcatidx]['id']==firstcatid #assert id - it should be the same
    keepidx_l.append(firstcatidx) #catidx to keep
    keepid_l.append(firstcatid) #catid to keep
keepidx_l=list(dict.fromkeys(keepidx_l)) # remove duplicates in the keep list
allidx_l=[index for index, value in enumerate(molajson['categories'])] # allidx in categories
removeidx_l=[idx for idx in allidx_l if idx not in keepidx_l] # remove idx 
removeitem_l=[newjson['categories'][removeidx] for removeidx in removeidx_l] #remove items #WARNING NECESSARY BECAUSE THE INDEX WILL CHANGE
print(len(allidx_l))
print(len(removeidx_l))
print(keepidx_l in removeidx_l)

100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2005.40it/s]

1261
1259
False


In [70]:
# REMOVE - newjson will be changed
for removeitem in removeitem_l: newjson['categories'].remove(removeitem)

In [71]:
print(newjson['categories'])

[{'supercategory': 'person', 'id': 1, 'name': 'nonaggressive', 'keypoints': ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle'], 'skeleton': [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13], [6, 12], [7, 13], [6, 7], [6, 8], [7, 9], [8, 10], [9, 11], [2, 3], [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7]], 'dataset': 1}, {'supercategory': 'kitchen', 'id': 43, 'name': 'aggressive', 'dataset': 1}]


In [72]:
# GET NEW IDs - #WARNING after remove
ct_l_id=[]
for i,c in enumerate(tqdm(newjson['categories'])):
    ct_l_id.append(c['id'])
newidx_l=[ct_l_id.index(id) for id in keepid_l] # make sure same sequence of keepid_l #SAME ORDER AS EXCEL
newid_l=[i+1 for i in range(len(keepid_l))] #same sequence of keepid_l
print(keepid_l)
print(newidx_l)
print(newid_l)

100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2008.77it/s]

[43, 1]
[1, 0]
[1, 2]


In [73]:
# SORT IDS - Excel order - newjson will be changed
categories_l=copy.copy(newjson['categories'])
for i,idx in enumerate(newidx_l):
    categories_l[idx]['id']=newid_l[i]
for i,idx in enumerate(newidx_l):
    newjson['categories'][i]=categories_l[idx] #TODO sort the id in the correct sequence

In [74]:
#TEST
print(categories_l)
print(newjson['categories'])
print(len(molajson['categories']))

[{'supercategory': 'person', 'id': 2, 'name': 'nonaggressive', 'keypoints': ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle'], 'skeleton': [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13], [6, 12], [7, 13], [6, 7], [6, 8], [7, 9], [8, 10], [9, 11], [2, 3], [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7]], 'dataset': 1}, {'supercategory': 'kitchen', 'id': 1, 'name': 'aggressive', 'dataset': 1}]
[{'supercategory': 'kitchen', 'id': 1, 'name': 'aggressive', 'dataset': 1}, {'supercategory': 'person', 'id': 2, 'name': 'nonaggressive', 'keypoints': ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle'], 'skeleton': [[16, 14], [14, 12], [1

### QUESTION: REMOVE HYPERPARAMETERS? Mantain only id and NAME? OR irrelevant?

#### change molajson['annotations']: [{category_id: , }] 
=> 1.get annotation idx from catid; 2.update annotations id ; 3. update newjson['annotations']

In [75]:
# 1.get annotation idx from catid
ann_catid_np=np.array(ann_catid)
classtomix_l_ann_catidx=[[np.where(ann_catid_np==id)[0].tolist()  for id in id_l] for id_l in classtomix_l_catid]
print(classtomix_l)
print(classtomix_l_catid)
print(keepid_l)
print(newid_l)
print(len(classtomix_l_ann_catidx[0]))

[['aggressive'], ['nonaggressive']]
[[43, 44, 45, 216, 256, 613, 675, 707, 905, 979, 1022, 1246, 1300], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 117, 118, 119, 120, 121, 122, 123, 124, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 160, 162, 163, 164, 165, 166, 167, 168, 169, 170, 172, 173, 174, 176, 177, 178, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 199, 200, 201, 202, 203, 204, 205, 207, 208, 209, 210, 211, 212, 214, 215, 217, 218, 220

In [76]:
#2.update annotations id & 3. update newjson['annotations']
newjson['annotations']=copy.copy(molajson['annotations']) #reset annotations
copy_ann_l=copy.copy(newjson['annotations'])
newjson['annotations']=[] #clear
for i, ann_catidx_l in enumerate(classtomix_l_ann_catidx): 
    for catidx_l in ann_catidx_l:
        for catidx in catidx_l:
            copy_ann_l[catidx]['category_id']=newid_l[i] # update catid
            newjson['annotations'].append(copy_ann_l[catidx]) #update newjson (annotations sequence id will be lost
print(len(newjson['annotations']))

133641


In [77]:
#TEST
print(molajson['annotations'][8875])
print(newjson['annotations'][8875])

{'segmentation': [[240.31, 208.35, 240.18, 206.03, 246.12, 202.03, 249.09, 203.71, 250.25, 206.03, 249.86, 208.48, 247.8, 214.03, 246.25, 218.55, 245.22, 220.36, 242.51, 221.0, 240.83, 217.13, 242.77, 211.58, 242.12, 210.03]], 'area': 118.00675, 'iscrowd': 0, 'image_id': 111398, 'bbox': [240.18, 202.03, 10.07, 18.97], 'category_id': 2, 'id': 704720, 'dataset': 1}
{'segmentation': [[188.07, 291.41, 184.23, 271.26, 195.75, 236.72, 213.02, 209.85, 239.89, 196.41, 271.55, 185.86, 292.66, 173.39, 295.54, 146.52, 314.73, 132.12, 342.56, 126.37, 362.71, 127.33, 379.98, 140.76, 388.62, 154.19, 380.94, 173.39, 377.1, 183.94, 372.31, 206.01, 370.39, 218.48, 362.71, 217.52, 353.11, 226.16, 345.44, 228.08, 337.76, 250.15, 356.95, 265.5, 337.76, 299.09, 350.24, 307.72, 355.99, 319.24, 352.16, 340.35, 342.56, 358.58, 336.8, 370.09, 297.46, 378.73, 274.43, 379.69, 256.2, 374.89, 254.28, 377.77, 231.25, 380.65, 215.9, 380.65, 216.86, 352.82, 244.69, 338.43, 264.84, 338.43, 281.15, 331.71, 285.95, 323.

### 4. Save mixed json

In [78]:
# fast
molajson['categories']=copy.copy(newjson['categories'])
molajson['annotations']=copy.copy(newjson['annotations'])

In [79]:
# save
print('\n >> SAVING...')
jsonfile=rdir+'test_aggressive.json'
with open(jsonfile, 'w') as f:
    json.dump(molajson, f)
print("JSON SAVED : {} \n".format(jsonfile))
for k in molajson:
    print(k, len(molajson[k]))


 >> SAVING...
JSON SAVED : D:/external_datasets/test_aggressive.json 

info 5
licenses 9
categories 2
videos 1488
images 177936
tracks 8132
segment_info 0
annotations 133641
datasets 2
